# LangSmith 데이터셋 생성

자체 RAG 평가용 데이터셋을 구축하는 방법에 대해 알아보겠습니다.

먼저, 데이터셋 구축을 위해서는 크게 3가지 과정에 대한 이해가 필요합니다.

Case: Retrieval 이 Question 에 Relevant 한지 평가

> Question - Retrieval

![](./assets/eval-03.png)

Case: Answer 이 Question 에 Relevant 한지 평가

> Question - Answer   

![](./assets/eval-04.png)

Case: Answer 가 Retrieval 된 문서 안에서 답변하였는지 (Hallucination Check)

> Retrieval - Answer

![](./assets/eval-05.png)

따라서, `Question`, `Retrieval`, `Answer` 의 3가지 정보가 필요한 것이 일반적이지만, `Retrieval` 에 대한 Ground Truth 구축이 사실상 어렵습니다.

만약, `Retrieval` 에 대한 Ground Truth 가 존재한다면, 모두 데이터셋으로 저장하여 활용하고, 그렇지 않다면 `Question`, `Answer` 만으로 데이터셋을 구축하여 활용할 수 있습니다.

In [ ]:
# 설치
# !pip install -qU langsmith langchain-teddynote

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

## 데이터셋 생성

`inputs` 와 `outputs` 를 활용하여 데이터셋을 생성합니다.

데이터셋은 `question` 과 `answer` 로 구성됩니다.

In [ ]:
import pandas as pd

# 질문과 답변 목록
inputs = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표한 날은 언제인가요?",
    "코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.",
]

# 질문에 대한 답변 목록
outputs = [
    "삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.",
    "2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.",
    "코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스 상태를 추적하고 투명성을 확보하기 위한 플랫폼입니다. 12개 기관과 협력하여 2,000여 개 데이터셋의 출처 정보를 제공하며, 개발자들이 데이터의 구성과 계보를 쉽게 파악할 수 있게 돕습니다.",
]

# 질문과 답변 쌍 생성
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]

# 데이터프레임으로 변환
df = pd.DataFrame(qa_pairs)

# 데이터프레임 출력
df.head()

혹은 이전의 튜토리얼에서 생성한 Synthetic Dataset 을 활용할 수 있습니다.

아래 코드는 업로드한 HuggingFace Dataset 을 활용하는 예시입니다.

(참고) 아래의 주석을 풀고 실행하여 `datasets` 라이브러리를 업데이트 후 진행해 주세요. 

In [ ]:
# !pip install -qU datasets

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
import os

# huggingface Dataset에서 repo_id로 데이터셋 다운로드
dataset = load_dataset(
    "teddylee777/rag-synthetic-dataset",  # 데이터셋 이름
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"],  # private 데이터인 경우 필요합니다.
)

# 데이터셋에서 split 기준으로 조회
huggingface_df = dataset["korean_v1"].to_pandas()
huggingface_df.head()

## LangSmith 테스트를 위한 Dataset 생성

- `Datasets & Testing` 에 새로운 데이터셋을 생성합니다.

![](./assets/eval-06.png)

csv 파일에서 LangSmith UI를 사용하여 직접 데이터셋을 생성할 수도 있습니다.

자세한 사항은 아래 문서를 참고해 주세요.

- [LangSmith UI 문서](https://docs.smith.langchain.com/evaluation/faq/manage-datasets)

In [ ]:
from langsmith import Client

client = Client()
dataset_name = "RAG_EVAL_DATASET"


# 데이터셋 생성 함수
def create_dataset(client, dataset_name, description=None):
    for dataset in client.list_datasets():
        if dataset.name == dataset_name:
            return dataset

    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description=description,
    )
    return dataset


# 데이터셋 생성
dataset = create_dataset(client, dataset_name)

# 생성된 데이터셋에 예제 추가
client.create_examples(
    inputs=[{"question": q} for q in df["question"].tolist()],
    outputs=[{"answer": a} for a in df["answer"].tolist()],
    dataset_id=dataset.id,
)

데이터셋에 예제를 나중에 추가할 수 있습니다.

In [ ]:
# 새로운 질문 목록
new_questions = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?",
]

# 새로운 답변 목록
new_answers = [
    "삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.",
    "사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자하고 향후 15억 달러를 추가로 투자하기로 했습니다.",
]

# UI에서 업데이트된 버전 확인
client.create_examples(
    inputs=[{"question": q} for q in new_questions],
    outputs=[{"answer": a} for a in new_answers],
    dataset_id=dataset.id,
)

축하합니다! 이제 데이터셋이 준비되었습니다.